# Tutorial: Build a Scalable Question Answering System

- **Level**: Beginner
- **Time to complete**: 20 minutes
- **Nodes Used**: `ElasticsearchDocumentStore`, `BM25Retriever`, `FARMReader`
- **Goal**: After completing this tutorial, you'll have built a scalable search system that runs on text files and can answer questions about Game of Thrones. You'll then be able to expand this system for your needs.


## Overview

Learn how to set up a question answering system that can search through complex knowledge bases and highlight answers to questions such as "Who is the father of Arya Stark?". In this tutorial, we'll work on a set of Wikipedia pages about Game of Thrones, but you can adapt it to search through internal wikis or a collection of financial reports, for example.

This tutorial introduces you to all the concepts needed to build such a question answering system. It also uses Haystack components, such as indexing pipelines, querying pipelines, and DocumentStores backed by external database services.

Let's learn how to build a question answering system and discover more about the marvelous seven kingdoms!


## Preparing the Colab Environment

- [Enable GPU Runtime](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)


## Installing Haystack

To start, let's install the latest release of Haystack with `pip`:

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]

Processing /home/dilab810/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734/langdetect-1.0.9-py3-none-any.whl
Processing /home/dilab810/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c/docopt-0.6.2-py2.py3-none-any.whl


ERROR: beir 2.0.0 has requirement elasticsearch==7.9.1, but you'll have elasticsearch 8.11.1 which is incompatible.


  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.5.0
    Uninstalling pydantic-2.5.0:
      Successfully uninstalled pydantic-2.5.0


### Enabling Telemetry 
Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [2]:
from haystack.telemetry import tutorial_running

tutorial_running(3)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set the logging level to INFO:

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Initializing the ElasticsearchDocumentStore

A DocumentStore stores the Documents that the question answering system uses to find answers to your questions. Here, we're using the [`ElasticsearchDocumentStore`](https://docs.haystack.deepset.ai/reference/document-store-api#module-elasticsearch) which connects to a running Elasticsearch service. It's a fast and scalable text-focused storage option. This service runs independently from Haystack and persists even after the Haystack program has finished running. To learn more about the DocumentStore and the different types of external databases that we support, see [DocumentStore](https://docs.haystack.deepset.ai/docs/document_store).

1. Download, extract, and set the permissions for the Elasticsearch installation image:

In [39]:
'''%%bash

#wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
#tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2'''

Password: su: Authentication failure


CalledProcessError: Command 'b'\n#wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q\n#tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz\nsu daemon -c "chown -R daemon:daemon elasticsearch-7.9.2"\n'' returned non-zero exit status 1.

2. Start the server:

In [5]:
'''%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch 말고

sudo docker ps 하기'''

If Docker is available in your environment (Colab notebooks do not support Docker), you can also start Elasticsearch using Docker. You can do this manually, or using our [`launch_es()`](https://docs.haystack.deepset.ai/reference/utils-api#module-doc_store) utility function.

In [ ]:
# from haystack.utils import launch_es

# launch_es()

3. Wait 30 seconds for the server to fully start up:

In [6]:
import time

time.sleep(30)

4. Initialize the ElasticsearchDocumentStore:


In [49]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
PASSWORD = "YOUR_OWN_PASSWORD"
document_store = ElasticsearchDocumentStore(host=host, username='elastic', password='PASSWORD', index="document")

ElasticsearchDocumentStore is up and running and ready to store the Documents.

## Indexing Documents with a Pipeline

The next step is adding the files to the DocumentStore. The indexing pipeline turns your files into Document objects and writes them to the DocumentStore. Our indexing pipeline will have two nodes: `TextConverter`, which turns `.txt` files into Haystack `Document` objects, and `PreProcessor`, which cleans and splits the text within a `Document`.

Once we combine these nodes into a pipeline, the pipeline will ingest `.txt` file paths, preprocess them, and write them into the DocumentStore.


1. Download 517 articles from the Game of Thrones Wikipedia. You can find them in *data/build_a_scalable_question_answering_system* as a set of *.txt* files.

In [50]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/build_a_scalable_question_answering_system"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip",
    output_dir=doc_dir,
)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip to 'data/build_a_scalable_question_answering_system'


True

2. Initialize the pipeline, TextConverter, and PreProcessor:

In [ ]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

To learn more about the parameters of the `PreProcessor`, see [Usage](https://docs.haystack.deepset.ai/docs/preprocessor#usage). To understand why document splitting is important for your question answering system's performance, see [Document Length](https://docs.haystack.deepset.ai/docs/optimization#document-length).

2. Add the nodes into an indexing pipeline. You should provide the `name` or `name`s of preceding nodes as the `input` argument. Note that in an indexing pipeline, the input to the first node is `File`.

In [57]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

3. Run the indexing pipeline to write the text data into the DocumentStore:

In [68]:
doc_dir = "data/build_a_scalable_question_answering_system"
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO - haystack.pipelines.base -  It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Preprocessing:   0%|          | 0/183 [00:00<?, ?docs/s]WARNING - haystack.nodes.preprocessor.preprocessor -  We found one or more sentences whose split count is higher than the split length.
WARNING - haystack.nodes.preprocessor.preprocessor -  Document dd048b8e5bcb7de1be5bd3937f15442f is 14232 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time. This document will be now hard-split at 10000 chars recursively.
Preprocessing: 100%|██████████| 183/183 [00:01<00:00, 167.93docs/s]


{'documents': [<Document: {'content': "\n\n'''Gendry''' is a fictional character in the ''A Song of Ice and Fire'' series of epic fantasy novels by American author George R. R. Martin, and its HBO television adaptation ''Game of Thrones''.\n\nFirst appearing in 1996's ''A Game of Thrones'', Gendry is a blacksmith apprentice in King's Landing, and an unacknowledged bastard of King Robert Baratheon.  He subsequently appeared in ''A Clash of Kings'' (1998), ''A Storm of Swords'' (2000) and ''A Feast for Crows'' (2005).  After Queen Cersei Lannister orders the execution of all of King Robert's bastards, Gendry is forced to flee King's Landing alongside Arya Stark under the protection of Yoren, a recruiter for the Night's Watch.  He later joins the outlaw group Brotherhood Without Banners and is knighted by its leader Beric Dondarrion, and becomes a follower of R'hllor.\n\nGendry is portrayed by English actor Joe Dempsie in the HBO television adaptation.\n\n== Character description ==\nGend

The code in this tutorial uses Game of Thrones data, but you can also supply your own `.txt` files and index them in the same way.

As an alternative, you can cast you text data into [Document objects](https://docs.haystack.deepset.ai/docs/documents_answers_labels#document) and write them into the DocumentStore using [`DocumentStore.write_documents()`](https://docs.haystack.deepset.ai/reference/document-store-api#basedocumentstorewrite_documents).

Now that the Documents are in the DocumentStore, let's initialize the nodes we want to use in our query pipeline.

## Initializing the Retriever

Our query pipeline is going to use a Retriever, so we need to initialize it. A Retriever sifts through all the Documents and returns only those that are relevant to the question. This tutorial uses the BM25Retriever. This is the recommended Retriever for a question answering system like the one we're creating. For more Retriever options, see [Retriever](https://docs.haystack.deepset.ai/docs/retriever).

In [69]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

The BM25Retriever is initialized and ready for the pipeline.

## Initializing the Reader

Our query pipeline also needs a Reader, so we'll initialize it next. A Reader scans the texts it received from the Retriever and extracts the top answer candidates. Readers are based on powerful deep learning models but are much slower than Retrievers at processing the same amount of text. This tutorials uses a FARMReader with a base-sized RoBERTa question answering model called [`deepset/roberta-base-squad2`](https://huggingface.co/deepset/roberta-base-squad2). It's a good all-round model to start with. To find a model that's best for your use case, see [Models](https://docs.haystack.deepset.ai/docs/reader#models).

In [70]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0, CUDA:1 - Number of GPUs: 2
INFO - haystack.modeling.utils -  Using devices: CUDA:0, CUDA:1 - Number of GPUs: 2
WARNING - haystack.modeling.infer -  Multiple devices are not supported in Inferencer, using the first device cuda:0.
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA:0, CUDA:1 - Number of GPUs: 2
WARNING - haystack.modeling.infer -  Multiple devices are not supported in QAInferencer inference, using the first device cuda:0.


## Creating the Retriever-Reader Pipeline

You can combine the Reader and Retriever in a querying pipeline using the `Pipeline` class. The combination of the two speeds up processing because the Reader only processes the Documents that it received from the Retriever. 

Initialize the `Pipeline` object and add the Retriever and Reader as nodes. You should provide the `name` or `name`s of preceding nodes as the input argument. Note that in a querying pipeline, the input to the first node is `Query`.

In [71]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

That's it! Your pipeline's ready to answer your questions!

## Asking a Question

1. Use the pipeline's `run()` method to ask a question. The query argument is where you type your question. Additionally, you can set the number of documents you want the Reader and Retriever to return using the `top-k` parameter. To learn more about setting arguments, see [Arguments](https://docs.haystack.deepset.ai/docs/pipelines#arguments). To understand the importance of the `top-k` parameter, see [Choosing the Right top-k Values](https://docs.haystack.deepset.ai/docs/optimization#choosing-the-right-top-k-values).


In [72]:
prediction = querying_pipeline.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:04<00:00,  4.82s/ Batches]


Here are some questions you could try out:
- Who is the father of Arya Stark?
- Who is the sister of Sansa?
- Who created the Dothraki vocabulary?

2. Print out the answers the pipeline returns:

In [73]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.993372917175293, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 207, 'end': 213}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_ids': ['9e3c863097d66aeed9992e0b6bf1f2f4'], 'meta': {'_split_id': 4, '_split_overlap': [{'doc_id': '241c8775e39c6c937c67bbd10ccc471c', 'range': [0, 266]}, {'doc_id': '87e8469dcf7354fd2a25fbd2ba07c543', 'range': [960, 1200]}]}}>,
             <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.9753615856170654, 'context': "k in the television series.\n\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's h", 'offsets_in_document': [{'start': 630, 'end': 633}], 'offsets_in_context': [{'start': 74, 'end': 77}], 'document_ids': ['7d3360fa29130e69ea6b2b

3. Simplify the printed answers:

In [88]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium` and `all`

'Query: Who is the father of Arya Stark?'
'Answers:'
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Ned',
        'context': 'k in the television series.\n'
                   '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's h"},
    {   'answer': 'Lord Eddard Stark',
        'context': 'rk daughters.\n'
                   '\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.'},
    {   'answer': 'Joffrey',
        'context': 'Mycah, sparring in the woods with broomsticks.  Arya '
    

In [89]:
print_answers(prediction, details="medium")  ## Choose from `minimum`, `medium` and `all`

'Query: Who is the father of Arya Stark?'
'Answers:'
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,',
        'score': 0.993372917175293},
    {   'answer': 'Ned',
        'context': 'k in the television series.\n'
                   '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's h",
        'score': 0.9753615856170654},
    {   'answer': 'Lord Eddard Stark',
        'context': 'rk daughters.\n'
                   '\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.',
        'score': 0.9566049575805664}

In [90]:
print_answers(prediction, details="all")  ## Choose from `minimum`, `medium` and `all`

'Query: Who is the father of Arya Stark?'
'Answers:'
[   <Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.993372917175293, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 207, 'end': 213}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_ids': ['9e3c863097d66aeed9992e0b6bf1f2f4'], 'meta': {'_split_id': 4, '_split_overlap': [{'doc_id': '241c8775e39c6c937c67bbd10ccc471c', 'range': [0, 266]}, {'doc_id': '87e8469dcf7354fd2a25fbd2ba07c543', 'range': [960, 1200]}]}}>,
    <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.9753615856170654, 'context': "k in the television series.\n\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's h", 'offsets_in_document': [{'start': 630, 'end': 633}], 'offsets_in_context': [{'start': 74, 'end': 77}], 'docu

And there you have it! Congratulations on building a scalable machine learning based question answering system!

# Next Steps

To learn how to improve the performance of the Reader, see [Fine-Tune a Reader](https://haystack.deepset.ai/tutorials/02_finetune_a_model_on_your_data).

In [75]:
prediction2 = querying_pipeline.run(
    query="Who is the sister of Sansa?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  8.78 Batches/s]


In [78]:
pprint(prediction2)

{'answers': [<Answer {'answer': 'Arya', 'type': 'extractive', 'score': 0.9572818279266357, 'context': "She doesn't have cool swordplay skills like her sister Arya; she isn't a smart seductress like Margaery Tyrell or a fierce queen like Cersei. She is t", 'offsets_in_document': [{'start': 55, 'end': 59}], 'offsets_in_context': [{'start': 55, 'end': 59}], 'document_ids': ['8d1fa41215fd69d25630886e871375c1'], 'meta': {'_split_id': 35, '_split_overlap': [{'doc_id': 'd7fc4d209e94f5fd6e7f01b4dd9e732e', 'range': [0, 231]}, {'doc_id': '20efbb043768c1d919702547478f2056', 'range': [835, 996]}]}}>,
             <Answer {'answer': 'Arya Stark', 'type': 'extractive', 'score': 0.9348183870315552, 'context': ' at the beginning of the series. She was raised with a younger sister Arya Stark, two younger brothers Rickon Stark and Bran Stark, as well as an olde', 'offsets_in_document': [{'start': 399, 'end': 409}], 'offsets_in_context': [{'start': 70, 'end': 80}], 'document_ids': ['4120a29e784c7b5d795f1

In [79]:
print_answers(prediction2, details="minimum")  ## Choose from `minimum`, `medium` and `all`

'Query: Who is the sister of Sansa?'
'Answers:'
[   {   'answer': 'Arya',
        'context': "She doesn't have cool swordplay skills like her sister "
                   "Arya; she isn't a smart seductress like Margaery Tyrell or "
                   'a fierce queen like Cersei. She is t'},
    {   'answer': 'Arya Stark',
        'context': ' at the beginning of the series. She was raised with a '
                   'younger sister Arya Stark, two younger brothers Rickon '
                   'Stark and Bran Stark, as well as an olde'},
    {   'answer': 'Myrcella',
        'context': 'sa is present when the royal family bids farewell to '
                   "Joffrey's sister, Myrcella, on her departure to Dorne to "
                   'form an alliance between the Lannisters '},
    {   'answer': 'Margaery',
        'context': 'Lannister-Tyrell forces, Loras Tyrell asks Joffrey to take '
                   'his sister, Margaery, as his bride. Joffrey accepts the '
                   'p

In [80]:
prediction3 = querying_pipeline.run(
    query="Who created the Dothraki vocabulary?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 12.33 Batches/s]


In [81]:
pprint(prediction3)

{'answers': [<Answer {'answer': 'David J. Peterson', 'type': 'extractive', 'score': 0.9771378636360168, 'context': 'e episode as the giant seen in the wildlings camp.\n\n===Valyrian===\nDavid J. Peterson, who created the Dothraki language for the first season of the sh', 'offsets_in_document': [{'start': 733, 'end': 750}], 'offsets_in_context': [{'start': 67, 'end': 84}], 'document_ids': ['43df89192697da27dff3c80fdd519cc6'], 'meta': {'_split_id': 5, '_split_overlap': [{'doc_id': 'da99838e256a6e49dceea240cabc92a1', 'range': [0, 281]}, {'doc_id': 'e9eee465a78da0d0f4be4d91c116f6bd', 'range': [718, 949]}]}}>,
             <Answer {'answer': 'David J. Peterson', 'type': 'extractive', 'score': 0.8763149976730347, 'context': "ge for ''Game of Thrones''\n\nThe Dothraki vocabulary was created by David J. Peterson well in advance of the adaptation. HBO hired the Language Creatio", 'offsets_in_document': [{'start': 915, 'end': 932}], 'offsets_in_context': [{'start': 67, 'end': 84}], 'document_ids

In [83]:
print_answers(prediction3, details="minimum")  ## Choose from `minimum`, `medium` and `all`

'Query: Who created the Dothraki vocabulary?'
'Answers:'
[   {   'answer': 'David J. Peterson',
        'context': 'e episode as the giant seen in the wildlings camp.\n'
                   '\n'
                   '===Valyrian===\n'
                   'David J. Peterson, who created the Dothraki language for '
                   'the first season of the sh'},
    {   'answer': 'David J. Peterson',
        'context': "ge for ''Game of Thrones''\n"
                   '\n'
                   'The Dothraki vocabulary was created by David J. Peterson '
                   'well in advance of the adaptation. HBO hired the Language '
                   'Creatio'},
    {   'answer': 'Dwight Schrute',
        'context': '\'s Ancestry" from the United States television show '
                   "''The Office'', Dwight Schrute created the Dothraki phrase "
                   '"throat rip" by placing "throat" in fro'},
    {   'answer': 'Daenerys',
        'context': 's were created.\n'
            

In [84]:
print_answers(prediction3, details="medium")  ## Choose from `minimum`, `medium` and `all`

'Query: Who created the Dothraki vocabulary?'
'Answers:'
[   {   'answer': 'David J. Peterson',
        'context': 'e episode as the giant seen in the wildlings camp.\n'
                   '\n'
                   '===Valyrian===\n'
                   'David J. Peterson, who created the Dothraki language for '
                   'the first season of the sh',
        'score': 0.9771378636360168},
    {   'answer': 'David J. Peterson',
        'context': "ge for ''Game of Thrones''\n"
                   '\n'
                   'The Dothraki vocabulary was created by David J. Peterson '
                   'well in advance of the adaptation. HBO hired the Language '
                   'Creatio',
        'score': 0.8763149976730347},
    {   'answer': 'Dwight Schrute',
        'context': '\'s Ancestry" from the United States television show '
                   "''The Office'', Dwight Schrute created the Dothraki phrase "
                   '"throat rip" by placing "throat" in fro',
        

In [85]:
print_answers(prediction3, details="all")  ## Choose from `minimum`, `medium` and `all`

'Query: Who created the Dothraki vocabulary?'
'Answers:'
[   <Answer {'answer': 'David J. Peterson', 'type': 'extractive', 'score': 0.9771378636360168, 'context': 'e episode as the giant seen in the wildlings camp.\n\n===Valyrian===\nDavid J. Peterson, who created the Dothraki language for the first season of the sh', 'offsets_in_document': [{'start': 733, 'end': 750}], 'offsets_in_context': [{'start': 67, 'end': 84}], 'document_ids': ['43df89192697da27dff3c80fdd519cc6'], 'meta': {'_split_id': 5, '_split_overlap': [{'doc_id': 'da99838e256a6e49dceea240cabc92a1', 'range': [0, 281]}, {'doc_id': 'e9eee465a78da0d0f4be4d91c116f6bd', 'range': [718, 949]}]}}>,
    <Answer {'answer': 'David J. Peterson', 'type': 'extractive', 'score': 0.8763149976730347, 'context': "ge for ''Game of Thrones''\n\nThe Dothraki vocabulary was created by David J. Peterson well in advance of the adaptation. HBO hired the Language Creatio", 'offsets_in_document': [{'start': 915, 'end': 932}], 'offsets_in_context': [{